In [ ]:
import os
import math
import numpy as np
import pandas as pd
from typing import List, Dict, Any

# Gymnasium imports
import gymnasium as gym
from gymnasium import spaces

# Scikit-Learn
from sklearn.preprocessing import StandardScaler

# stable-baselines3 (PPO)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback

import torch.nn as nn
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 120)

In [ ]:
# Simple data loading - no complex feature engineering
# This simple approach previously achieved MAE 34

print("Using simple feature set without advanced engineering.")
print("This configuration previously achieved MAE 34.")

In [ ]:
# STEP 3: Simple HVAC Environment (Previously achieved MAE 34)

class HVACValveEnv(gym.Env):
    """Simple HVAC Valve control environment that achieved MAE 34"""

    metadata = {"render_modes": ["human"], "render_fps": 30}

    def __init__(
        self,
        df: pd.DataFrame,
        obs_cols: List[str],
        scaler=None,
        episode_length: int = 256,
        start_index: int = 0,
        smoothness_penalty_coef: float = 0.05,  # Simple proven coefficient
        energy_penalty_coef: float = 0.0005,    # Simple proven coefficient
    ):
        super().__init__()
        self.df = df.copy()
        self.obs_cols = obs_cols
        self.scaler = scaler
        self.episode_length = episode_length
        self.start_index = start_index
        self.smoothness_penalty_coef = smoothness_penalty_coef
        self.energy_penalty_coef = energy_penalty_coef

        # Ensure valid indices
        max_start = len(self.df) - episode_length - 1
        if self.start_index > max_start:
            self.start_index = max_start

        # Action space: continuous control of valve opening [0, 100]
        self.action_space = spaces.Box(low=0.0, high=100.0, shape=(1,), dtype=np.float32)

        # Observation space: simple features only
        n_features = len(obs_cols)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(n_features,), dtype=np.float32
        )

        self.reset()

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        # Random start within valid range
        valid_range = min(1000, len(self.df) - self.episode_length - 10)
        if valid_range > 0:
            self.current_idx = np.random.randint(0, valid_range)
        else:
            self.current_idx = 0
        
        self.step_count = 0
        self.prev_action = 50.0  # Initialize to middle value
        
        obs = self._get_observation()
        info = self._get_info()
        return obs, info

    def _get_observation(self):
        """Get current observation (scaled features)"""
        if self.current_idx >= len(self.df):
            self.current_idx = len(self.df) - 1
        
        row = self.df.iloc[self.current_idx]
        obs = row[self.obs_cols].values.astype(np.float32)
        
        # Apply scaling if available
        if self.scaler is not None:
            obs = self.scaler.transform(obs.reshape(1, -1)).flatten().astype(np.float32)
        
        return obs

    def _get_info(self):
        """Get info dictionary"""
        if self.current_idx >= len(self.df):
            return {"error": 0.0, "gt_valve": 50.0}
        
        row = self.df.iloc[self.current_idx]
        gt_valve = float(row.get("Valve", 50.0))
        
        return {
            "gt_valve": gt_valve,
            "timestep": self.current_idx,
            "episode_step": self.step_count
        }

    def step(self, action):
        # Ensure action is a scalar
        action_val = float(action[0]) if hasattr(action, '__len__') else float(action)
        action_val = np.clip(action_val, 0.0, 100.0)
        
        # Get ground truth
        if self.current_idx >= len(self.df):
            self.current_idx = len(self.df) - 1
        
        row = self.df.iloc[self.current_idx]
        gt_valve = float(row.get("Valve", 50.0))
        
        # Calculate error
        error = abs(action_val - gt_valve)
        
        # Simple reward calculation - this achieved MAE 34
        reward_primary = -error  # Main reward: minimize error
        
        # Energy penalty (prefer moderate valve positions)
        energy_penalty = -self.energy_penalty_coef * (action_val - 50.0) ** 2
        
        # Smoothness penalty (avoid rapid changes)
        smoothness_penalty = -self.smoothness_penalty_coef * abs(action_val - self.prev_action)
        
        # Total reward
        reward = reward_primary + energy_penalty + smoothness_penalty
        
        # Update state
        self.prev_action = action_val
        self.step_count += 1
        self.current_idx += 1
        
        # Episode termination
        done = (self.step_count >= self.episode_length) or (self.current_idx >= len(self.df))
        truncated = False
        
        # Get next observation and info
        if not done:
            obs = self._get_observation()
        else:
            obs = np.zeros_like(self._get_observation())
        
        info = self._get_info()
        info["error"] = error
        
        return obs, reward, done, truncated, info

    def render(self, mode="human"):
        pass

    def close(self):
        pass

print("Simple HVACValveEnv created (previously achieved MAE 34)")
print("Key parameters:")
print("- smoothness_penalty_coef: 0.05")
print("- energy_penalty_coef: 0.0005")
print("- Simple reward: -error + energy_penalty + smoothness_penalty")

In [ ]:
# STEP 4: Simple PPO Setup (Previously achieved MAE 34)

# Load dataframe
feature_df = pd.read_csv("./files/final_feature_df.csv")
print("Loaded dataframe shape:", feature_df.shape)

# Simple observation columns - exactly what achieved MAE 34
obs_cols = [
    "RaTemp", "SaTemp", "RaHumidity", "ThermEnergy", 
    "main_temp", "main_humidity", "wind_speed", "clouds_all", 
    "hour_sin", "hour_cos", "minute_sin", "minute_cos", "dayofweek", "is_weekend","Occp",
    "delta_supply_return", "delta_outdoor_indoor", "delta_humidity","Valve_lag_1", "RaTemp_lag_1", "SaTemp_lag_1", 
    "RaHumidity_lag_1"
]

# Fill NA and fit scaler (simple approach)
df_for_scaler = feature_df[obs_cols].fillna(method="ffill").fillna(0.0)
scaler = StandardScaler()
scaler.fit(df_for_scaler.values)

# Simple environment factory - no curriculum learning
def make_env(start_index=0):
    def _init():
        env = HVACValveEnv(
            df=feature_df.fillna(method="ffill").fillna(0.0),
            obs_cols=obs_cols,
            scaler=scaler,
            episode_length=256,  # Simple fixed episode length
            start_index=start_index,
            smoothness_penalty_coef=0.05,   # Proven working values
            energy_penalty_coef=0.0005,     # Proven working values
        )
        return env
    return _init

# Simple vectorized environment (single environment)
vec_env = DummyVecEnv([make_env()])
vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=False, clip_obs=10.0)

# Simple PPO model with proven hyperparameters
model = PPO(
    "MlpPolicy",
    vec_env,
    verbose=1,
    learning_rate=1e-4,                  # Proven learning rate
    batch_size=128,                      # Proven batch size
    policy_kwargs=dict(net_arch=[512, 256]),  # Proven architecture
)

print("Simple PPO model created with proven configuration:")
print("- Learning rate: 1e-4")
print("- Batch size: 128") 
print("- Network architecture: [512, 256]")
print("- Single environment (no parallelization)")
print("- This exact configuration previously achieved MAE 34")

In [ ]:
# Display observation columns for reference
print("Observation columns used in the model:")
for i, col in enumerate(obs_cols, 1):
    print(f"  {i:2d}. {col}")
print(f"\\nTotal features: {len(obs_cols)}")

In [ ]:
# STEP 5: Simple Training (Previously achieved MAE 34)

# Setup simple callbacks
checkpoint_callback = CheckpointCallback(save_freq=10000, save_path="./checkpoints/", name_prefix="ppo_hvac_simple")

# Create simple eval environment
eval_env = DummyVecEnv([make_env()])
eval_env = VecNormalize(eval_env, norm_obs=True, norm_reward=False, clip_obs=10.0)
eval_env.obs_rms = vec_env.obs_rms  # Sync normalization stats

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path="./best_model/",
    n_eval_episodes=5,
    eval_freq=10000,
    deterministic=True
)

# Simple proven training parameters
TOTAL_TIMESTEPS = 3_000_000  # This achieved MAE 34

print(f"Starting simple training with {TOTAL_TIMESTEPS:,} timesteps")
print("This exact configuration previously achieved MAE 34")
print("\\nBeginning training...")

# Start training
model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=[checkpoint_callback, eval_callback])

# Save final model and VecNormalize statistics
os.makedirs("./models", exist_ok=True)
model.save("./models/ppo_hvac_simple_final")
vec_env.save("./models/vec_normalize_simple.pkl")

print("\\nTraining completed! Model saved as 'ppo_hvac_simple_final'")

In [ ]:
# STEP 6: Next Steps for Incremental MAE Reduction

print("=== TRAINING COMPLETED WITH SIMPLE CONFIGURATION ===")
print("Current configuration previously achieved MAE 34")
print()
print("=== INCREMENTAL IMPROVEMENT STRATEGIES ===")
print()
print("✅ PROVEN WORKING CONFIGURATION:")
print("   - Learning rate: 1e-4")
print("   - Batch size: 128")
print("   - Network: [512, 256]")
print("   - Smoothness penalty: 0.05")
print("   - Energy penalty: 0.0005")
print("   - Result: MAE 34")
print()
print("🎯 NEXT EXPERIMENTS TO TRY (one at a time):")
print()
print("1. **HYPERPARAMETER FINE-TUNING:**")
print("   - learning_rate: [5e-5, 2e-4, 3e-4]")
print("   - batch_size: [64, 256]")
print("   - smoothness_penalty_coef: [0.02, 0.1]")
print("   - Expected improvement: 1-3 MAE points")
print()
print("2. **LONGER TRAINING:**")
print("   - Try 5M or 10M timesteps")
print("   - Monitor for overfitting")
print("   - Expected improvement: 1-2 MAE points")
print()
print("3. **ENSEMBLE METHODS:**")
print("   - Train 3-5 models with different seeds")
print("   - Average their predictions")
print("   - Expected improvement: 2-4 MAE points")
print()
print("4. **BEHAVIORAL CLONING PRE-TRAINING:**")
print("   - Pre-train with supervised learning first")
print("   - Then fine-tune with RL")
print("   - Expected improvement: 3-5 MAE points")
print()
print("5. **ARCHITECTURAL VARIATIONS:**")
print("   - Try [256, 128], [1024, 512]")
print("   - Test deeper: [512, 256, 128]")
print("   - Expected improvement: 1-2 MAE points")
print()
print("⚠️  AVOID THESE (previously made MAE worse):")
print("   - Complex reward engineering (increased MAE from 34 to 45)")
print("   - Curriculum learning")
print("   - Adaptive rewards")
print("   - Enhanced observation spaces")
print()
print("🎯 REALISTIC TARGETS:")
print("   - Conservative: MAE 30-32")
print("   - Optimistic: MAE 25-28")
print("   - Best case with ensemble: MAE 20-25")

In [ ]:
# STEP 6 START: Evaluate PPO performance
# ---------------------------------------------------------------------------
# Comments: We'll run the trained policy over a portion of the dataframe and record actions.

# Reload model + vec normalize if necessary
# model = PPO.load1("./models/ppo_hvac_final")
# vec_env = VecNormalize.load("./models/vec_normalize.pkl", DummyVecEnv([make_env(start_index=0)]))
# vec_env.reset()

# For evaluation, use an un-normalized instance of the environment and manually apply scaler
eval_df = feature_df.copy().fillna(method="ffill").fillna(0.0)
start_eval = 0
end_eval = min(2000, len(eval_df) - 1)
eval_episode_length = end_eval - start_eval

eval_env_simple = HVACValveEnv(df=eval_df, obs_cols=obs_cols, scaler=scaler, episode_length=eval_episode_length, start_index=start_eval)
obs, _ = eval_env_simple.reset()

pred_actions = []
true_vals = []
infos = []

done = False
while not done:
    # SB3 models expect actions in their raw range; since our action space is [0,100], model will output that.
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _, info = eval_env_simple.step(action)
    pred_actions.append(float(action[0]))
    true_vals.append(float(info.get("gt_valve", np.nan)))
    infos.append(info)



# Plot true vs predicted
plt.figure(figsize=(12, 4))
plt.plot(true_vals, label="True Valve")
plt.plot(pred_actions, label="Predicted Valve (Agent)")
plt.legend()
plt.title("True vs Predicted Valve over evaluation segment")
plt.show()

# Save evaluation results
eval_res = pd.DataFrame({"true_valve": true_vals, "pred_valve": pred_actions})
eval_res.to_csv("./models/evaluation_results.csv", index=False)

# ---------------------------------------------------------------------------
# REPORT (brief): How to improve results
# ---------------------------------------------------------------------------
# 1) Increase TOTAL_TIMESTEPS and tune hyperparameters (learning rate, n_steps, batch_size).
# 2) Adjust reward coefficients: smoothness_penalty_coef and energy_penalty_coef.
# 3) Add richer state: previous Valve, time-of-day cyclic features, occupancy as categorical.
# 4) Use curriculum learning: start with short episodes and increase length.
# 5) Use model ensembles or offline behavioral cloning as warm-start from dataset actions.
# 6) Consider action scaling: if Valve behaves as ratio, map to [0,1] internally for better learning.

In [ ]:
# Compute metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(true_vals, pred_actions)
mae = mean_absolute_error(true_vals, pred_actions)
print(f"Evaluation MSE: {mse:.4f}, MAE: {mae:.4f}")

In [ ]:
# STEP 7: Model Diagnostics and Analysis

def analyze_model_performance(true_vals, pred_actions, infos):
    """Analyze model performance in detail"""
    true_vals = np.array(true_vals)
    pred_actions = np.array(pred_actions)
    
    # Basic metrics
    mae = np.mean(np.abs(true_vals - pred_actions))
    mse = np.mean((true_vals - pred_actions)**2)
    rmse = np.sqrt(mse)
    
    # Error distribution analysis
    errors = np.abs(true_vals - pred_actions)
    error_percentiles = np.percentile(errors, [25, 50, 75, 90, 95, 99])
    
    print(f"=== Model Performance Analysis ===")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"Error Percentiles: 25%={error_percentiles[0]:.2f}, 50%={error_percentiles[1]:.2f}, ")
    print(f"                  75%={error_percentiles[2]:.2f}, 90%={error_percentiles[3]:.2f}, ")
    print(f"                  95%={error_percentiles[4]:.2f}, 99%={error_percentiles[5]:.2f}")
    
    # Identify problematic ranges
    high_error_mask = errors > np.percentile(errors, 90)
    if np.any(high_error_mask):
        print(f"\nHigh error instances ({np.sum(high_error_mask)} cases):")
        print(f"True values range: {true_vals[high_error_mask].min():.2f} - {true_vals[high_error_mask].max():.2f}")
        print(f"Predictions range: {pred_actions[high_error_mask].min():.2f} - {pred_actions[high_error_mask].max():.2f}")
    
    # Plot error distribution
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.hist(errors, bins=50, alpha=0.7)
    plt.xlabel('Absolute Error')
    plt.ylabel('Frequency')
    plt.title('Error Distribution')
    
    plt.subplot(1, 3, 2)
    plt.scatter(true_vals, pred_actions, alpha=0.5, s=1)
    plt.plot([true_vals.min(), true_vals.max()], [true_vals.min(), true_vals.max()], 'r--')
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    plt.title('True vs Predicted Scatter')
    
    plt.subplot(1, 3, 3)
    residuals = pred_actions - true_vals
    plt.scatter(true_vals, residuals, alpha=0.5, s=1)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('True Values')
    plt.ylabel('Residuals')
    plt.title('Residuals Plot')
    
    plt.tight_layout()
    plt.show()
    
    return mae, mse, rmse

# Run analysis if we have evaluation results
if 'true_vals' in locals() and 'pred_actions' in locals():
    mae, mse, rmse = analyze_model_performance(true_vals, pred_actions, infos)
else:
    print("Run evaluation first to analyze performance")

In [ ]:
# Additional Techniques for Better Performance

# 1. Ensemble Prediction (using multiple checkpoints)
def load_and_predict_ensemble(obs, checkpoint_paths):
    """Load multiple models and average their predictions"""
    predictions = []
    for path in checkpoint_paths:
        try:
            model_temp = PPO.load(path)
            pred, _ = model_temp.predict(obs, deterministic=True)
            predictions.append(pred[0])
        except:
            continue
    return np.mean(predictions) if predictions else 50.0  # fallback

# 2. Behavioral Cloning Pretraining (optional)
def pretrain_with_behavioral_cloning():
    """Pre-train the policy using supervised learning on the dataset"""
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.neural_network import MLPRegressor
    
    # Prepare training data
    X = feature_df[obs_cols].fillna(method="ffill").fillna(0.0).values
    X_scaled = scaler.transform(X)
    y = feature_df['Valve'].fillna(method="ffill").fillna(50.0).values
    
    # Train a supervised model
    bc_model = MLPRegressor(
        hidden_layer_sizes=(512, 256, 128),
        learning_rate_init=0.001,
        max_iter=500,
        early_stopping=True,
        validation_fraction=0.2,
        n_iter_no_change=20
    )
    
    # Split data
    split_idx = int(0.8 * len(X_scaled))
    bc_model.fit(X_scaled[:split_idx], y[:split_idx])
    
    print(f"Behavioral cloning model trained. Score: {bc_model.score(X_scaled[split_idx:], y[split_idx:]):.4f}")
    return bc_model

# 3. Advanced Feature Engineering
def create_advanced_features(df):
    """Create additional features that might help"""
    df_enhanced = df.copy()
    
    # Rolling statistics
    window_size = 10
    for col in ['RaTemp', 'SaTemp', 'RaHumidity', 'Valve_lag_1']:
        if col in df_enhanced.columns:
            df_enhanced[f'{col}_rolling_mean'] = df_enhanced[col].rolling(window=window_size).mean()
            df_enhanced[f'{col}_rolling_std'] = df_enhanced[col].rolling(window=window_size).std()
    
    # Temperature differences and ratios
    if 'RaTemp' in df_enhanced.columns and 'SaTemp' in df_enhanced.columns:
        df_enhanced['temp_ratio'] = df_enhanced['RaTemp'] / (df_enhanced['SaTemp'] + 1e-8)
        df_enhanced['temp_change_rate'] = df_enhanced['RaTemp'].diff()
    
    # Interaction features
    if 'Occp' in df_enhanced.columns and 'main_temp' in df_enhanced.columns:
        df_enhanced['occupancy_temp_interaction'] = df_enhanced['Occp'] * df_enhanced['main_temp']
    
    return df_enhanced.fillna(method='ffill').fillna(0.0)

print("Advanced techniques defined. To use:")
print("1. Run pretrain_with_behavioral_cloning() before PPO training")
print("2. Use ensemble prediction for final evaluation")
print("3. Enhanced features can improve observation space")